## Importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Data Loading

In [2]:
PROJECT_DIR = r"C:\Projects\AWS-SageMaker-Flight_Price_Predictor"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    return pd.read_csv(file_path)

In [6]:
flights = get_data("flight_price")
flights.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features.
- Colums `Route` and `Total_Stops` have a missing value in each.
- Data types of some features are not appropriate.

## Prelimnary Analysis

### Data Types 

In [11]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [12]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [17]:
flights.Date_of_Journey.iloc[12]

'24/06/2019'

In [18]:
flights.Dep_Time.iloc[12]

'11:40'

### Duplicates

In [21]:
flights.duplicated().sum()

np.int64(220)

In [26]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### Observations

- The data type of `Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be changed to datetime.
- The data type of `Duration` and `Total_Stops` is mixed. it should be numeric Type.
- The dataset has 220 duplicate rows. These should be removed.

## Detailed Analysis

### Airline

In [50]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some airlines have inconsistent/inaccurate values

In [58]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### Date_of_Journey

In [72]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [74]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [83]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [84]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_Time

In [96]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [95]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [99]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\Monil Shah\AppData\Local\Temp\ipykernel_23644\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time

In [102]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [110]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

- There are no special characters they are just dates

### Duration

In [116]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [124]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()    
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [126]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]  
)

6474    5m
Name: Duration, dtype: object

In [128]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- This observation indexed 6474 has duration of 5 minutes, which is clearly wrong and since we have only one such observation so it's better to delete it.

In [156]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour = lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [160]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour = lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename("duration_minutes")      #for validation
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### Total_Stops

In [165]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [182]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)  #with regex last s becomes optional because of ?
    .pipe(lambda ser: pd.to_numeric(ser))
    .unique()
)

array([ 0.,  2.,  1.,  3., nan,  4.])

### Additional_Info

In [186]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [189]:
flights.Additional_Info.replace("No info","No Info").unique()

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

### BEST practices

- Method chaining
- Do not over write existing columns instead create a new one 


In [46]:
flights.select_dtypes(include="O").columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

## Cleaning Opearations

In [161]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand=True)
        .set_axis(["hour","minute"],axis=1)
        .assign(
            hour = lambda df_: (
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minute = lambda df_: (
                df_
                .minute
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )
    
        

In [193]:
def clean_data(df):
    return(
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include="O").columns
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_: (
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()
            ),
            date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey,dayfirst=True),
            dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_: df_.additional_info.replace("No info","No Info")
        )
        .drop(columns="route")
    )

In [195]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\Monil Shah\AppData\Local\Temp\ipykernel_23644\3624115277.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
C:\Users\Monil Shah\AppData\Local\Temp\ipykernel_23644\3624115277.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


## Splitting Data 

In [205]:
## If need sample the dataset to reduce training time on sage maker since we are doing for learning purpose only
# flights_cleaned.sample(1000)

In [197]:
X = flights_cleaned.drop(columns="price")
y = flights_cleaned.price.copy()

In [198]:
X_, X_test, y_, y_test = train_test_split(X,y,test_size=0.2,random_state=9)
X_train, X_val, y_train, y_val = train_test_split(X_,y_,test_size=0.2,random_state=9)

In [204]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

(6695, 9) (6695,)
(1674, 9) (1674,)
(2093, 9) (2093,)


## Exporting subsets

In [206]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    
    X.join(y).to_csv(file_path,index=False)

    return pd.read_csv(file_path).head()

In [207]:
export_data(X_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-03,Delhi,Cochin,13:15:00,09:25:00,1210,2.0,No Info,10467
1,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,04:25:00,550,1.0,In-flight meal not included,12898
2,Jet Airways,2019-05-21,Kolkata,Banglore,06:30:00,19:50:00,800,1.0,No Info,14781
3,Jet Airways,2019-05-18,Delhi,Cochin,11:00:00,14:15:00,195,0.0,No Info,9564
4,Air India,2019-05-15,Kolkata,Banglore,14:15:00,16:45:00,150,0.0,No Info,5510


In [208]:
export_data(X_val,y_val,"validation")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-12,Chennai,Kolkata,05:15:00,07:40:00,145,0.0,No Info,3540
1,Spicejet,2019-03-24,Chennai,Kolkata,08:20:00,10:35:00,135,0.0,No Info,3859
2,Air Asia,2019-05-09,Kolkata,Banglore,19:55:00,23:30:00,215,1.0,No Info,5162
3,Indigo,2019-05-27,Chennai,Kolkata,13:15:00,15:35:00,140,0.0,No Info,3597
4,Air India,2019-05-09,Delhi,Cochin,06:05:00,09:25:00,1640,1.0,No Info,7480


In [209]:
export_data(X_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-05-24,Kolkata,Banglore,10:00:00,01:20:00,920,2.0,No Info,14083
1,Indigo,2019-03-12,Kolkata,Banglore,21:25:00,00:05:00,160,0.0,No Info,4148
2,Multiple Carriers,2019-03-09,Delhi,Cochin,07:30:00,19:45:00,735,1.0,No Info,12615
3,Air India,2019-05-21,Mumbai,Hyderabad,21:05:00,22:25:00,80,0.0,No Info,3100
4,Jet Airways,2019-05-01,Kolkata,Banglore,21:10:00,10:05:00,775,1.0,In-flight meal not included,10844
